# Features for modeling labeled points

## Import modules & libraries, initialize EE

In [1]:
import math
import geemap
import ee

from areaOfInterestMask import semiarid
from featuresForModeling import generateFeatures

ee.Initialize()

## Calculate features

### Seasonality using a phenology model

In [2]:
l8srSeasonalityParams = generateFeatures.seasonalityParamsL8(returnExisting = True)

# rgb visualisation as sanity check
# Instead of harmonicLS8sr.select('NDVI').median().multiply(1) as band 3 in original notebook,
# using l8srSeasonalityParams.select('amplitude').multiply(5), because 
# harmonicLS8sr NDVI band is used only for viz here, so not exposed in generateFeatures
rgb = ee.Image.cat(
    l8srSeasonalityParams.select('phase').unitScale(-math.pi, math.pi),
    l8srSeasonalityParams.select('amplitude').multiply(5),
    l8srSeasonalityParams.select('amplitude').multiply(5)).hsvToRgb()

Map = geemap.Map()
Map.addLayer(rgb, {}, 'Phenology')
Map

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

### Tasselled cap coefficients

In [3]:
tct_1719 = generateFeatures.tasselledCapCoeffsL8(returnExisting = True)
diff = tct_1719.select('tct_diff_seg')

Map.addLayer(diff, {}, 'tct_difference')

### Inter-percentile differences in multi-temporal NDVI

In [4]:
ndviMultitemporalStats = generateFeatures.multiTemporalInterPercentileDifferencesL8(returnExisting = True)
print(ndviMultitemporalStats.bandNames().getInfo())

nd_grbr = ndviMultitemporalStats.select('nd_grbr').float()
Map.addLayer(nd_grbr.clamp(0,1), {}, 'nd_grbr')

['mn', 'md', 'mx', 'grng_ext', 'brng_ext', 'nd_grbr', 'md_seg', 'brng_ext_seg', 'grng_ext_seg', 'nd_grbr_seg']


### PALSAR aggregation

In [5]:
palsarMedian = generateFeatures.palsarAggregation(returnExisting = True)

Map.addLayer(palsarMedian.clamp(0,1), {}, 'palsar')

### Evapo Transpiration aggregation

In [6]:
etAnn = generateFeatures.evapotranspirationAggregation(returnExisting = True)

Map.addLayer(etAnn.clamp(0,1), {}, 'et')

### Elevation aggregation

In [7]:
elev = generateFeatures.elevationAggregation(returnExisting = True)
print(elev.bandNames().getInfo())
Map.addLayer(elev.clamp(0,1), {}, 'elev_seg')

['elev_seg']


### Precipitation aggregation

In [8]:
ppt = generateFeatures.precipitationAggregation(returnExisting = True)
print(ppt.bandNames().getInfo())
Map.addLayer(ppt.clamp(0,1), {}, 'ppt_seg')

['annrf_seg']


### Topography (MTPI) aggregation

In [9]:
topo_mtpi = generateFeatures.topographyMtpiAggregation(returnExisting = True)
print(topo_mtpi.bandNames().getInfo())
Map.addLayer(topo_mtpi.clamp(0,1), {}, 'topo_mtpi_seg')

['topo_seg']


### Topography (HAND) aggregation

In [10]:
topo_hand = generateFeatures.topographyHandAggregation(returnExisting = True)
print(topo_hand.bandNames().getInfo())
Map.addLayer(topo_hand.clamp(0,1), {}, 'topo_hand_seg')

['topo_hand_seg']


### Geomorphology Terrain Ruggedness Index (TRI) aggregation

In [ ]:
geom_tri = generateFeatures.geomorphologyTerrainRuggednessAggregation(returnExisting = True)
print(geom_tri.bandNames().getInfo())
Map.addLayer(geom_tri.clamp(0,1), {}, 'geom_tri_seg')

## Assemble all features (and/or their segmented versions) into a composite and export

In [ ]:
sampledPoints = generateFeatures.assembleFeatureBandsAndExport(startFreshExport = True)
print(sampledPoints.first().propertyNames().getInfo())
Map.addLayer(sampledPoints, {}, 'sampled points')